In [1]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time

warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    indexes.reverse()
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Financials")
	probabilityThreshold = 0.7
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [2]:
def poolBuild(trainIndexes):
    t = time.time()
    featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
    forest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = trainIndexes, target= 'Rate of Return', features = featureList, \
                                               featureLength = 12, targetLength = 3, sector = "Financials", percentileTarget = 90, percentileAvoid = 10)
    print("Build time: " + str(time.time()-t) + " seconds.")
    return forest

In [13]:
indexes = []
for i in rebalanceIndexes(4,28):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-240, -237, -234, -231, -228, -225, -222, -219, -216, -213, -210,
       -207, -204, -201, -198, -195, -192, -189, -186, -183, -180, -177,
       -174, -171, -168, -165, -162, -159, -156, -153, -150, -147, -144,
       -141, -138, -135, -132, -129, -126, -123, -120, -117, -114, -111,
       -108, -105, -102,  -99,  -96,  -93,  -90,  -87,  -84,  -81,  -78,
        -75,  -72,  -69,  -66,  -63,  -60,  -57,  -54,  -51,  -48,  -45,
        -42]), array([-237, -234, -231, -228, -225, -222, -219, -216, -213, -210, -207,
       -204, -201, -198, -195, -192, -189, -186, -183, -180, -177, -174,
       -171, -168, -165, -162, -159, -156, -153, -150, -147, -144, -141,
       -138, -135, -132, -129, -126, -123, -120, -117, -114, -111, -108,
       -105, -102,  -99,  -96,  -93,  -90,  -87,  -84,  -81,  -78,  -75,
        -72,  -69,  -66,  -63,  -60,  -57,  -54,  -51,  -48,  -45,  -42,
        -39]), array([-234, -231, -228, -225, -222, -219, -216, -213, -210, -207, -204,
       -201, -198,

In [25]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8
Finished fitting.
Build time: 1917.4678819179535 seconds.
Finished fitting.
Build time: 1918.3827109336853 seconds.
Finished fitting.
Build time: 1919.1819767951965 seconds.
Finished fitting.
Finished fitting.
Build time: 1921.5957760810852 seconds.
Build time: 1921.8411450386047 seconds.
Finished fitting.
Finished fitting.
Build time: 1922.6303181648254 seconds.
Finished fitting.
Build time: 1922.8115742206573 seconds.
Build time: 1922.9709379673004 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False), RandomForestClassifier(bootstrap=True, class_weight='balanced',
            cri

In [26]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,28), forestList))
print(returnsList)

# of multiprocess cpus: 8
('AGM', 'CFR', 'CMA', 'EZPW', 'FBP', 'FFIN', 'HBHC', 'IBKC', 'MSL', 'OFG', 'PB', 'RF', 'SF', 'SMMF', 'TCF', 'TSBK', 'WRLD', 'ZION')
-24
-22
AGM: 0.2260361280302603
CFR: 0.28898520797982874
CMA: 0.27337055805852506
EZPW: 0.5346768395881707
FBP: 0.37515975861283524
FFIN: 0.20379822194044817
HBHC: 0.11840895348899494
IBKC: 0.21285591525788927
MSL: 0.27997702607737573
OFG: 0.4168547528725224
PB: 0.26587620391307576
RF: 0.221013625056385
SF: 0.12786099543773277
SMMF: 0.34799446607147333
TCF: 0.18467035411570976
TSBK: 0.11327079986228217
WRLD: 0.16663513997868362
ZION: 0.2552674137674993
Total Return: 0.2562617977838718
('EZPW', 'OFG', 'UBFO')
-21
-19
EZPW: 0.2898920181025837
OFG: 0.1491365671598328
UBFO: 0.11864555025007317
Total Return: 0.18589137850416323
('AMTD', 'CASH', 'CMA', 'ETFC', 'EZPW', 'KEY', 'LBAI', 'OCN', 'OFG', 'PJC', 'STBA')
-18
-16
AMTD: 0.0401097372739736
CASH: 0.17402839526671432
CMA: 0.09667413510131606
ETFC: 0.06529638400067306
EZPW: -0.05971923

In [14]:
indexes = []
for i in rebalanceIndexes(28,52):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-264, -261, -258, -255, -252, -249, -246, -243, -240, -237, -234,
       -231, -228, -225, -222, -219, -216, -213, -210, -207, -204, -201,
       -198, -195, -192, -189, -186, -183, -180, -177, -174, -171, -168,
       -165, -162, -159, -156, -153, -150, -147, -144, -141, -138, -135,
       -132, -129, -126, -123, -120, -117, -114, -111, -108, -105, -102,
        -99,  -96,  -93,  -90,  -87,  -84,  -81,  -78,  -75,  -72,  -69,
        -66]), array([-261, -258, -255, -252, -249, -246, -243, -240, -237, -234, -231,
       -228, -225, -222, -219, -216, -213, -210, -207, -204, -201, -198,
       -195, -192, -189, -186, -183, -180, -177, -174, -171, -168, -165,
       -162, -159, -156, -153, -150, -147, -144, -141, -138, -135, -132,
       -129, -126, -123, -120, -117, -114, -111, -108, -105, -102,  -99,
        -96,  -93,  -90,  -87,  -84,  -81,  -78,  -75,  -72,  -69,  -66,
        -63]), array([-258, -255, -252, -249, -246, -243, -240, -237, -234, -231, -228,
       -225, -222,

In [27]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8
Finished fitting.
Build time: 2359.0850019454956 seconds.
Finished fitting.
Build time: 2360.033859014511 seconds.
Finished fitting.
Build time: 2360.9837822914124 seconds.
Finished fitting.
Finished fitting.
Build time: 2364.325914144516 seconds.
Finished fitting.
Build time: 2365.0530722141266 seconds.
Build time: 2366.5277881622314 seconds.
Finished fitting.
Build time: 2370.418027162552 seconds.
Finished fitting.
Build time: 2381.578246116638 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False), RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criteri

In [28]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(28,52), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

# of multiprocess cpus: 8
('FNHC', 'FRBK', 'LTS')
-48
-46
FNHC: 0.20175780757035877
FRBK: 0.18322694390876704
LTS: -0.02173998663640586
Total Return: 0.12108158828090665
('HIFS', 'LTS', 'SBSI')
-45
-43
HIFS: 0.12618893732777803
LTS: 0.032470385030784144
SBSI: 0.12677838227064564
Total Return: 0.09514590154306928
('GSBC', 'HALL', 'SBCF', 'SMMF', 'UVE', 'WETF')
-42
-40
GSBC: 0.182479422450609
HALL: 0.1996230264297072
SBCF: 0.19589285550843805
SMMF: 0.10809747220659949
UVE: 0.23319388716771128
WETF: 0.22060440479553334
Total Return: 0.1899818447597664
('MKTX', 'UVE', 'WETF')
-39
-37
MKTX: 0.14722025123960236
UVE: 0.18017148071303568
WETF: 0.13698165481667557
Total Return: 0.15479112892310454
('CACC', 'CASH', 'DHIL', 'DNBF', 'INBK', 'OPY', 'OZRK')
-36
-34
CACC: 0.253554748917602
CASH: 0.13706453257050688
DHIL: 0.25833288439655533
DNBF: 0.16851864618089873
INBK: 0.2947082201895328
OPY: 0.12418692085569738
OZRK: 0.057340546615245014
Total Return: 0.18481521424657688
('WSFS',)
-33
-31
WSFS: 0

In [29]:
indexes = []
for i in rebalanceIndexes(52,76):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-288, -285, -282, -279, -276, -273, -270, -267, -264, -261, -258,
       -255, -252, -249, -246, -243, -240, -237, -234, -231, -228, -225,
       -222, -219, -216, -213, -210, -207, -204, -201, -198, -195, -192,
       -189, -186, -183, -180, -177, -174, -171, -168, -165, -162, -159,
       -156, -153, -150, -147, -144, -141, -138, -135, -132, -129, -126,
       -123, -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,
        -90]), array([-285, -282, -279, -276, -273, -270, -267, -264, -261, -258, -255,
       -252, -249, -246, -243, -240, -237, -234, -231, -228, -225, -222,
       -219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186, -183, -180, -177, -174, -171, -168, -165, -162, -159, -156,
       -153, -150, -147, -144, -141, -138, -135, -132, -129, -126, -123,
       -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,  -90,
        -87]), array([-282, -279, -276, -273, -270, -267, -264, -261, -258, -255, -252,
       -249, -246,

In [30]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8
Finished fitting.
Build time: 2364.7827479839325 seconds.
Finished fitting.
Build time: 2369.015805244446 seconds.
Finished fitting.
Build time: 2379.7602078914642 seconds.
Finished fitting.
Build time: 2380.8794779777527 seconds.
Finished fitting.
Build time: 2395.046727180481 seconds.
Finished fitting.
Build time: 2396.778678894043 seconds.
Finished fitting.
Build time: 2402.083676099777 seconds.
Finished fitting.
Build time: 2406.5073432922363 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=5,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=False,
            random_state=None, verbose=0, warm_start=False), RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criteri

In [31]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(52,76), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

# of multiprocess cpus: 8
('ANCX', 'BFIN', 'FBIZ', 'GBNK', 'HCI', 'HIIQ', 'HTH', 'PFBC', 'TREE', 'WSBF')
-72
-70
ANCX: 0.20258824159142286
BFIN: 0.164463939393948
FBIZ: 0.23498193706898407
GBNK: 0.20409535634351528
HCI: 0.25314760936256686
HIIQ: -0.06919185666321237
HTH: -0.038347308274623515
PFBC: 0.20677265265811906
TREE: 0.04604393850140687
WSBF: 0.2845402776546675
Total Return: 0.14890947876367946
('BFIN', 'GNW', 'HCI', 'HTH', 'LTXB', 'PRAA', 'TREE', 'UCFC', 'WSBF')
-69
-67
BFIN: 0.14780950448881103
GNW: -0.03891541624967365
HCI: 0.28355267910976956
HTH: 0.010521378741532583
LTXB: 0.14360344955359539
PRAA: 0.20317974211150158
TREE: 0.3582241309255618
UCFC: 0.2523774870908523
WSBF: -0.17073793581799923
Total Return: 0.13217944666155015
('BOFI', 'FAF', 'GNW', 'MRLN', 'PHH', 'TBBK', 'VRTS', 'WD', 'WSBF')
-66
-64
BOFI: 0.1773552464800372
FAF: 0.16601566297068437
GNW: 0.11925223520705042
MRLN: 0.3117256408843665
PHH: 0.17609399175710694
TBBK: 0.16194999829724166
VRTS: 0.1124233246341894

In [32]:
indexes = []
for i in rebalanceIndexes(76,100):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-312, -309, -306, -303, -300, -297, -294, -291, -288, -285, -282,
       -279, -276, -273, -270, -267, -264, -261, -258, -255, -252, -249,
       -246, -243, -240, -237, -234, -231, -228, -225, -222, -219, -216,
       -213, -210, -207, -204, -201, -198, -195, -192, -189, -186, -183,
       -180, -177, -174, -171, -168, -165, -162, -159, -156, -153, -150,
       -147, -144, -141, -138, -135, -132, -129, -126, -123, -120, -117,
       -114]), array([-309, -306, -303, -300, -297, -294, -291, -288, -285, -282, -279,
       -276, -273, -270, -267, -264, -261, -258, -255, -252, -249, -246,
       -243, -240, -237, -234, -231, -228, -225, -222, -219, -216, -213,
       -210, -207, -204, -201, -198, -195, -192, -189, -186, -183, -180,
       -177, -174, -171, -168, -165, -162, -159, -156, -153, -150, -147,
       -144, -141, -138, -135, -132, -129, -126, -123, -120, -117, -114,
       -111]), array([-306, -303, -300, -297, -294, -291, -288, -285, -282, -279, -276,
       -273, -270,

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8


In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(76,100), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [19]:
indexes = []
for i in rebalanceIndexes(100,124):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-336, -333, -330, -327, -324, -321, -318, -315, -312, -309, -306,
       -303, -300, -297, -294, -291, -288, -285, -282, -279, -276, -273,
       -270, -267, -264, -261, -258, -255, -252, -249, -246, -243, -240,
       -237, -234, -231, -228, -225, -222, -219, -216, -213, -210, -207,
       -204, -201, -198, -195, -192, -189, -186, -183, -180, -177, -174,
       -171, -168, -165, -162, -159, -156, -153, -150, -147, -144, -141,
       -138]), array([-333, -330, -327, -324, -321, -318, -315, -312, -309, -306, -303,
       -300, -297, -294, -291, -288, -285, -282, -279, -276, -273, -270,
       -267, -264, -261, -258, -255, -252, -249, -246, -243, -240, -237,
       -234, -231, -228, -225, -222, -219, -216, -213, -210, -207, -204,
       -201, -198, -195, -192, -189, -186, -183, -180, -177, -174, -171,
       -168, -165, -162, -159, -156, -153, -150, -147, -144, -141, -138,
       -135]), array([-330, -327, -324, -321, -318, -315, -312, -309, -306, -303, -300,
       -297, -294,

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(100,124), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [20]:
indexes = []
for i in rebalanceIndexes(124,148):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-360, -357, -354, -351, -348, -345, -342, -339, -336, -333, -330,
       -327, -324, -321, -318, -315, -312, -309, -306, -303, -300, -297,
       -294, -291, -288, -285, -282, -279, -276, -273, -270, -267, -264,
       -261, -258, -255, -252, -249, -246, -243, -240, -237, -234, -231,
       -228, -225, -222, -219, -216, -213, -210, -207, -204, -201, -198,
       -195, -192, -189, -186, -183, -180, -177, -174, -171, -168, -165,
       -162]), array([-357, -354, -351, -348, -345, -342, -339, -336, -333, -330, -327,
       -324, -321, -318, -315, -312, -309, -306, -303, -300, -297, -294,
       -291, -288, -285, -282, -279, -276, -273, -270, -267, -264, -261,
       -258, -255, -252, -249, -246, -243, -240, -237, -234, -231, -228,
       -225, -222, -219, -216, -213, -210, -207, -204, -201, -198, -195,
       -192, -189, -186, -183, -180, -177, -174, -171, -168, -165, -162,
       -159]), array([-354, -351, -348, -345, -342, -339, -336, -333, -330, -327, -324,
       -321, -318,

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(124,148), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [23]:
indexes = []
for i in rebalanceIndexes(148,172):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-384, -381, -378, -375, -372, -369, -366, -363, -360, -357, -354,
       -351, -348, -345, -342, -339, -336, -333, -330, -327, -324, -321,
       -318, -315, -312, -309, -306, -303, -300, -297, -294, -291, -288,
       -285, -282, -279, -276, -273, -270, -267, -264, -261, -258, -255,
       -252, -249, -246, -243, -240, -237, -234, -231, -228, -225, -222,
       -219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186]), array([-381, -378, -375, -372, -369, -366, -363, -360, -357, -354, -351,
       -348, -345, -342, -339, -336, -333, -330, -327, -324, -321, -318,
       -315, -312, -309, -306, -303, -300, -297, -294, -291, -288, -285,
       -282, -279, -276, -273, -270, -267, -264, -261, -258, -255, -252,
       -249, -246, -243, -240, -237, -234, -231, -228, -225, -222, -219,
       -216, -213, -210, -207, -204, -201, -198, -195, -192, -189, -186,
       -183]), array([-378, -375, -372, -369, -366, -363, -360, -357, -354, -351, -348,
       -345, -342,

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(148,172), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)

In [24]:
indexes = []
for i in rebalanceIndexes(172,196):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-408, -405, -402, -399, -396, -393, -390, -387, -384, -381, -378,
       -375, -372, -369, -366, -363, -360, -357, -354, -351, -348, -345,
       -342, -339, -336, -333, -330, -327, -324, -321, -318, -315, -312,
       -309, -306, -303, -300, -297, -294, -291, -288, -285, -282, -279,
       -276, -273, -270, -267, -264, -261, -258, -255, -252, -249, -246,
       -243, -240, -237, -234, -231, -228, -225, -222, -219, -216, -213,
       -210]), array([-405, -402, -399, -396, -393, -390, -387, -384, -381, -378, -375,
       -372, -369, -366, -363, -360, -357, -354, -351, -348, -345, -342,
       -339, -336, -333, -330, -327, -324, -321, -318, -315, -312, -309,
       -306, -303, -300, -297, -294, -291, -288, -285, -282, -279, -276,
       -273, -270, -267, -264, -261, -258, -255, -252, -249, -246, -243,
       -240, -237, -234, -231, -228, -225, -222, -219, -216, -213, -210,
       -207]), array([-402, -399, -396, -393, -390, -387, -384, -381, -378, -375, -372,
       -369, -366,

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(172,196), forestList))
print(returnsListTemp)
returnsList.extend(returnsListTemp)